# Screen Time Mental Health PCA

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.decomposition import PCA

pd.set_option('max_row',2000)
pd.set_option('max_column',300)

base_dir = os.path.join(os.getcwd(), os.pardir)

Get the data.

In [3]:
data_path = os.path.join(base_dir, "data/altered/maps-feature-mapping.csv")
data_df = pd.read_csv(data_path)
data_df

,X,comp_bed_9,mat_dep,mat_age,weight_16,height_16,iq,comp_noint_bed_16,comp_int_bed_16,talk_phon_wend,text_wend,talk_mob_wend,comp_wend,musi_wend,read_wend,work_wend,alon_wend,draw_wend,play_wend,tv_wend,out_win_wend,out_sum_wend,tran_wend,talk_phon_week,text_week,talk_mob_week,comp_week,musi_week,read_week,work_week,alon_week,draw_week,play_week,tv_week,out_win_week,out_sum_week,tran_week,pat_pres_10,pat_pres_8,pat_pres,num_home,mat_anx_1,mat_anx_18m,mat_anx_8m,agg_score,emot_cruel,phys_cruel,mat_anx_0m,pat_ses,mat_ses,pat_edu,mat_edu,parity,dep_band_15,dep_band_13,dep_band_10,dep_band_07,anx_band_15,anx_band_13,anx_band_10,anx_band_07,exercise,child_bull,phone_14_wend,phone_14_week,musi_13,tv_bed_9,own_mob,has_dep_diag,secd_diag,prim_diag,panic_score,dep_thoughts,dep_score,comp_house,tv_bed_16,creat_14,comp_games,fam_tv_eve,fam_tv_aft,fam_tv_mor,sex,birth_order
0,1,NaN,3.0,30.0,59.294132,181.602831,107.0,NaN,NaN,1.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,1.0,0.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,4.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,3.0,2.0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0
1,2,NaN,9.0,26.0,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
2,3,NaN,3.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.0,0.0,0.0,3.0,5.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,1.0,0.0
3,4,0.0,NaN,22.0,49.812426,160.224186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,1.0,2.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,0.0
4,5,0.0,8.0,31.0,62.270030,191.703227,132.0,1.0,NaN,0.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,0.0,1.0,2.0,1.0,0.0,1.0,NaN,NaN,0.0,4.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,3.0,5.0,2.0,2.0,1.0,0.0,0.0,0.0,NaN,2.0,1.0,1.0,NaN,3.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,2.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13729,13730,NaN,13.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,0.0,0.0
13730,13731,0.0,6.0,29.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,3.0,3.0,1.0,1.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,3.0,2.0,0.0,1.0,1.0,1.0,1.0,5.0,0.0,1.0,NaN,9.0,0.0,0.0,0.0,1.0,1.0,4.0,4.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,0.0,1.0,0.0
13731,13732,NaN,21.0,37.0,58.075832,169.235565,108.0,NaN,NaN,1.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,3.0,3.0,1.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,3.0,3.0,1.0,2.0,1.0,1.0,1.0,4.0,1.0,0.0,0.0,6.0,1.0,0.0,0.0,2.0,2.0,4.0,4.0,0.0,4.0,0.0,NaN,0.0,3.0,1.0,2.0,1.0,4.0,0.0,1.0,2.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0
13732,13733,0.0,10.0,30.0,NaN,NaN,103.


#### The percentage of NaNs in each column

In [4]:
d1 =data_df.isnull().sum()/len(data_df)*100
d1.sort_values(ascending = False)

comp_noint_bed_16    96.570555
comp_house           94.808504
dep_thoughts         86.857434
comp_int_bed_16      81.287316
tv_bed_16            71.770788
own_mob              68.064657
dep_score            67.139945
prim_diag            67.139945
secd_diag            67.139945
has_dep_diag         67.139945
panic_score          67.139945
out_sum_wend         64.890054
musi_wend            64.824523
work_wend            64.809961
talk_mob_wend        64.802679
read_wend            64.802679
comp_wend            64.766273
text_wend            64.751711
draw_wend            64.744430
out_win_wend         64.693461
play_wend            64.686180
alon_wend            64.686180
talk_phon_wend       64.686180
tv_wend              64.664337
tran_wend            64.606087
exercise             63.805155
child_bull           63.674093
read_week            63.644969
talk_phon_week       63.601281
musi_week            63.572157
text_week            63.564875
out_win_week         63.543032
alon_wee



#### Replace the NaNs with -1

In [5]:
for c in data_df:
    data_df[c] = data_df[c].fillna(-1)
data_df

,X,comp_bed_9,mat_dep,mat_age,weight_16,height_16,iq,comp_noint_bed_16,comp_int_bed_16,talk_phon_wend,text_wend,talk_mob_wend,comp_wend,musi_wend,read_wend,work_wend,alon_wend,draw_wend,play_wend,tv_wend,out_win_wend,out_sum_wend,tran_wend,talk_phon_week,text_week,talk_mob_week,comp_week,musi_week,read_week,work_week,alon_week,draw_week,play_week,tv_week,out_win_week,out_sum_week,tran_week,pat_pres_10,pat_pres_8,pat_pres,num_home,mat_anx_1,mat_anx_18m,mat_anx_8m,agg_score,emot_cruel,phys_cruel,mat_anx_0m,pat_ses,mat_ses,pat_edu,mat_edu,parity,dep_band_15,dep_band_13,dep_band_10,dep_band_07,anx_band_15,anx_band_13,anx_band_10,anx_band_07,exercise,child_bull,phone_14_wend,phone_14_week,musi_13,tv_bed_9,own_mob,has_dep_diag,secd_diag,prim_diag,panic_score,dep_thoughts,dep_score,comp_house,tv_bed_16,creat_14,comp_games,fam_tv_eve,fam_tv_aft,fam_tv_mor,sex,birth_order
0,1,-1.0,3.0,30.0,59.294132,181.602831,107.0,-1.0,-1.0,1.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,1.0,0.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,4.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,3.0,2.0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,4.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0
1,2,-1.0,9.0,26.0,-1.000000,-1.000000,85.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,9.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0
2,3,-1.0,3.0,24.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,0.0,0.0,0.0,3.0,5.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,0.0,1.0,0.0
3,4,0.0,-1.0,22.0,49.812426,160.224186,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0,1.0,1.0,2.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,1.0,0.0
4,5,0.0,8.0,31.0,62.270030,191.703227,132.0,1.0,-1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,0.0,1.0,2.0,1.0,0.0,1.0,-1.0,-1.0,0.0,4.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,3.0,5.0,2.0,2.0,1.0,0.0,0.0,0.0,-1.0,2.0,1.0,1.0,-1.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13729,13730,-1.0,13.0,27.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,9.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,2.0,2.0,0.0,0.0
13730,13731,0.0,6.0,29.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,3.0,3.0,1.0,1.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,3.0,2.0,0.0,1.0,1.0,1.0,1.0,5.0,0.0,1.0,-1.0,9.0,0.0,0.0,0.0,1.0,1.0,4.0,4.0,2.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,3.0,0.0,1.0,1.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,2.0,0.0,0.0,1.0,0.0
13731,13732,-1.0,21.0,37.0,58.075832,169

In [6]:
train_data=np.array(data_df)
train_data

array([[ 1.0000e+00, -1.0000e+00,  3.0000e+00, ...,  2.0000e+00,
         0.0000e+00,  0.0000e+00],
       [ 2.0000e+00, -1.0000e+00,  9.0000e+00, ..., -1.0000e+00,
         0.0000e+00,  1.0000e+00],
       [ 3.0000e+00, -1.0000e+00,  3.0000e+00, ...,  0.0000e+00,
         1.0000e+00,  0.0000e+00],
       ...,
       [ 1.3732e+04, -1.0000e+00,  2.1000e+01, ...,  0.0000e+00,
         1.0000e+00,  0.0000e+00],
       [ 1.3733e+04,  0.0000e+00,  1.0000e+01, ...,  2.0000e+00,
         1.0000e+00,  0.0000e+00],
       [ 1.3734e+04,  0.0000e+00,  6.0000e+00, ...,  2.0000e+00,
         1.0000e+00,  0.0000e+00]])

---
#### Use PCA to reduce dimensionality

In [7]:
def meanX(dataX):
    return np.mean(dataX,axis=0) 


def pca_method(XMat, k):
    average = meanX(XMat)
    m, n = np.shape(XMat)
    data_adjust = []
    avgs = np.tile(average, (m, 1))
    data_adjust = XMat - avgs
    covX = np.cov(data_adjust.T)   
    featValue, featVec=  np.linalg.eig(covX)  
    index = np.argsort(-featValue) 
    finalData = []
    if k > n:
        print ("k must lower than feature number")
        return
    else:
        selectVec = np.matrix(featVec.T[index[:k]]) 
        finalData = data_adjust * selectVec.T
    return finalData

In [8]:
pca_mat=pca_method(train_data,50)
print(pca_mat)
print()
print(np.shape(pca_mat))

[[-6.86647890e+03 -1.33579104e+02 -6.14131450e-01 ... -3.90782498e-01
   1.16842485e+00 -9.27702624e-01]
 [-6.86551662e+03  5.27528334e+01 -5.42569448e+01 ... -8.41052193e-02
  -7.22878261e-03 -1.14602718e-03]
 [-6.86451006e+03  8.59565817e+01  2.49692448e+01 ... -3.71123937e-01
  -1.13859393e-01  3.66353608e-03]
 ...
 [ 6.86451846e+03 -1.20847869e+02 -8.34294307e+00 ...  4.89399479e-02
  -3.81822402e-02  4.80476693e-01]
 [ 6.86548173e+03  4.72765990e+01 -7.33972388e+01 ... -1.04104000e+00
   1.26774809e+00 -1.26494281e+00]
 [ 6.86652081e+03 -1.28515824e+02  2.08693119e+00 ...  2.15710918e-01
   1.26474949e-01  3.69451474e-01]]

(13734, 50)



#### sklearn inbuilt PCA function

In [9]:
dataMat = np.array(data_df) 
pca = PCA(n_components=50) # or '=mle'
new_data=pca.fit_transform(dataMat)
print(np.shape(new_data))
print()
print(new_data,end='\n')
print()
print(pca.explained_variance_ratio_)

(13734, 50)

[[-6.86647890e+03  1.33579104e+02  6.14131450e-01 ...  4.46922046e-01
   1.28097007e+00 -7.48221164e-01]
 [-6.86551662e+03 -5.27528334e+01  5.42569448e+01 ...  8.98905731e-02
  -7.61993801e-03 -2.60493823e-03]
 [-6.86451006e+03 -8.59565817e+01 -2.49692448e+01 ...  3.67300129e-01
  -1.13192521e-01 -6.34624545e-03]
 ...
 [ 6.86451846e+03  1.20847869e+02  8.34294307e+00 ...  9.03915661e-02
  -1.43567637e-01  5.26438088e-01]
 [ 6.86548173e+03 -4.72765990e+01  7.33972388e+01 ...  1.10588598e+00
   1.48270696e+00 -9.42323279e-01]
 [ 6.86652081e+03  1.28515824e+02 -2.08693119e+00 ... -2.19367585e-01
   5.73003173e-02  3.37322267e-01]]

[9.99301111e-01 5.69760286e-04 1.15427893e-04 3.50196666e-06
 2.74707371e-06 1.87391618e-06 1.50931881e-06 1.04262174e-06
 4.01104625e-07 3.83900073e-07 3.33433655e-07 1.96879436e-07
 1.74045396e-07 1.33162363e-07 1.15812725e-07 9.91743891e-08
 7.70190454e-08 7.15896828e-08 6.28746966e-08 6.09140449e-08
 5.14222709e-08 4.46392200e-08 4.35009518e-08